- Introduction to the ABSESpy package
- Context and motivation (Explain Hotelling's Law)
- Model setup
- Model level methods
- Agent level methods
- Visualizations and model metrics

# Beginner Tutorial: Introducing the ABSESpy package

```
%pip install --upgrade abses"
```

In [1]:
import numpy as np

from abses import Actor, MainModel

# Instantiate a random number generator
rng = np.random.default_rng()


# Calculate euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

In [2]:
class Shop(Actor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.price = 10
        self.area_count = 0
        self.id = rng.integers(
            1e6, 1e7
        )  # FIXME: no two agents should have the same id

    def __repr__(self):
        return f"Shop {self.id}"

    def step(self):
        self.adjust_price()
        self.adjust_position()

    def advance(self):
        self.affect_price()
        self.affect_position()

    def adjust_price(self):
        pass

    def adjust_position(self):
        pass

    def affect_price(self):
        pass

    def affect_position(self):
        pass


class Hotelling(MainModel):
    def __init__(self, N, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_agents = N

    def setup(self):
        # Initialize a grid of shape (12, 12)
        self.nature.create_module(how="from_resolution", shape=(12, 12))

        # Create a list of agents
        self.agents.create(Shop, self.num_agents)

        # Placed agents on the grid randomly
        positions = rng.integers(12, size=(self.num_agents, 2), dtype=np.int8)

        for i, agent in enumerate(self.actors):
            agent.put_on_layer(
                layer=self.nature.major_layer, pos=tuple(positions[i])
            )

    def step(self):
        # recalculate areas and assign them to each agent
        areas = self.recalculate_areas()

        for shop in self.actors:
            shop.area_count = areas[shop]

        # trigger all agents to step
        self.actors.trigger("step")
        # trigger all agents to advance and, thus, update their prices and positions
        self.actors.trigger("advance")

    def update(self):
        # get all the data from agents (prices, areas, revenue)
        self.actors

    def recalculate_areas(self):
        areas = {}

        for shop in self.actors:
            areas[shop] = 0

        _width = self.nature.major_layer.width  # columns
        _height = self.nature.major_layer.height  # rows

        for i in range(_height):
            for j in range(_width):
                dist = {}
                for shop in self.actors:
                    _dist = euclidean_distance(i, j, shop.pos[0], shop.pos[1])
                    dist[shop] = _dist
                _choice = min(dist, key=dist.get)
                areas[_choice] += 1

        return areas

In [3]:
# INITIALIZE = True

# parameters = {
# "time" : {
#     "start" : "0",
#     "end" : "100",
#     }
# }

# if INITIALIZE:
#     model = Hotelling(N=2, parameters=parameters)

# model.run_model()

model = Hotelling(N=2)

model.setup()

2023-11-06 20:05:31.246 | DEBUG    | abses.time:_parse_time_settings:220 - start_dt: 2023-11-06T20:05:31.246246
2023-11-06 20:05:31.250 | DEBUG    | abses.time:_parse_time_settings:224 - end_dt: None
2023-11-06 20:05:31.252 | DEBUG    | abses.time:_parse_time_settings:228 - duration: None
2023-11-06 20:05:31.258 | DEBUG    | abses.time:_parse_time_settings:232 - irregular: None
2023-11-06 20:05:31.260 | DEBUG    | abses.time:_parse_time_settings:233 - Ticking mode: tick


In [4]:
model.step()

{Shop 5589255: 68, Shop 2626677: 76}


In [5]:
model.actors[0].pos

(7, 4)

In [6]:
model.actors[1].pos

(4, 6)

In [7]:
model.actors[0]

Shop 5589255

In [8]:
class Shop(Actor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.price = 10
        self.area_count = 0
        self.id = rng.integers(
            1e6, 1e7
        )  # FIXME: no two agents should have the same id

    def step(self):
        self.adjust_price()
        self.adjust_location()

    def advance(self):
        self.affect_price()
        self.affect_location()

    def adjust_price(self):
        pass

    def adjust_location(self):
        pass

    def affect_price(self):
        pass

    def affect_location(self):
        pass

In [9]:
class Hotelling(MainModel):
    def __init__(self, N, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_agents = N

    def setup(self):
        self.agents.create(Shop, num=self.num_agents)

        self.nature.create_module(how="from_resolution", shape=(12, 12))

    def step(self):
        areas = self.recalculate_areas()

        for shop in self.agents.to_list():
            shop.area_count = areas[shop]

        self.agents.step()

        print("Step: ", self.steps)

    def update(self):
        self.all_agents.record("area_count")

    # Calculate areas for which each agent is a monopoly
    def recalculate_areas(self):
        areas = {}

        for shop in self.agents.to_list():
            areas[shop] = 0

        xs = self.nature.grid.shape[0]
        ys = self.nature.grid.shape[1]

        for i in range(xs):
            for j in range(ys):
                dist = {}
                for shop in self.agents.to_list():
                    _dist = euclid_dist(i, j, shop.pos[0], shop.pos[1])
                    dist[shop] = _dist
                _choice = min(dist, key=dist.get)
                areas[_choice] += 1

        return areas

In [ ]:
model = Hotelling(N=2)

2023-11-05 10:35:42.438 | DEBUG    | abses.time:_parse_time_settings:220 - start_dt: 2023-11-05T10:35:42.438738
2023-11-05 10:35:42.438 | DEBUG    | abses.time:_parse_time_settings:224 - end_dt: None
2023-11-05 10:35:42.449 | DEBUG    | abses.time:_parse_time_settings:228 - duration: None
2023-11-05 10:35:42.451 | DEBUG    | abses.time:_parse_time_settings:232 - irregular: None
2023-11-05 10:35:42.455 | DEBUG    | abses.time:_parse_time_settings:233 - Ticking mode: tick


In [ ]:
model.run_model()

AttributeError: 'BaseNature' object has no attribute 'grid'

<ActorsList: (10)Shop>